<a href="https://colab.research.google.com/github/rikchik/Text-summarization-using-T5/blob/main/notebook1f5fa6000f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'news-summary:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1895%2F791838%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241010%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241010T160248Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2c9aae7c01dc11a10646a1eb1e631d060d293bde86810db149734aaedb29dac0741b53e70239b0a476699aef9aa6bd6b61a3c06c0715d0be1a690ad9e109835a56204a34856ba581243a200fb16fc9d653afe10dfbaf30d9c58deda9f8c7cff77a43b111f38dd2801512a2ca923c98ad30605d607bd4da0d5b8ac9a1879f25cb031678a271ae0e5ff56b44a288121144c91219935be0dde6087f40c1eec873e0c3382403f12527e7a02e589ced329595784abe0e8e40aaa0b255f4869e49dd024d34e51beda103da4ead4fdde4a2bced21928021dd05535e49ab66dc48b2f156e399500c96f43e90682ab1ef10027415e223ccc18a9ec9c8f0e1b5bb6a3abeff'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast
import torch
import seaborn as sns

In [ ]:
data=pd.read_csv("/kaggle/input/news-summary/news_summary_more.csv",encoding="latin-1")
data=pd.read_csv("/kaggle/input/news-summary/news_summary.csv",encoding="latin-1")

In [ ]:
data.iloc[0]["ctext"]

In [ ]:
data.iloc[0]["text"]

In [ ]:
data.head(5)

In [ ]:
data=data[["ctext","text"]]

In [ ]:
data=data.dropna()

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
test_data=data.iloc[:677].reset_index()[["ctext","text"]]
train_data=data.iloc[677:].reset_index()[["ctext","text"]]

In [ ]:
test_data.head()

In [ ]:
mx,mn=0,20000
for i in range(len(train_data)):
    mx=max(len(train_data.iloc[i]["ctext"].split(" ")),mx)
    mn=min(len(train_data.iloc[i]["ctext"].split(" ")),mn)

print(f"Range summary: {mn}-{mx}")

In [ ]:
mx,mn=0,20000
for i in range(len(train_data)):
    mx=max(len(train_data.iloc[i]["text"].split(" ")),mx)
    mn=min(len(train_data.iloc[i]["text"].split(" ")),mn)

print(f"Range summary: {mn}-{mx}")

In [ ]:
train_data["ctext_len"]=train_data['ctext'].apply(lambda x:len(x.split()))
train_data["text_len"]=train_data["text"].apply(lambda x:len(x.split()))

In [ ]:
train_data["ctext_len"]

In [ ]:
sns.histplot(train_data["ctext_len"],bins=50)

In [ ]:
sns.histplot(train_data["text_len"],bins=15)

In [ ]:
class NewsDataset(Dataset):
    def __init__(self,data1,tokenizer,max_elength,max_olength):
        self.data=data1
        self.tokenizer=tokenizer
        self.max_elength=max_elength
        self.max_olength=max_olength

    def __len__(self):
        return len(self.data)

    def __getitem__(self,id):

        text=self.data.iloc[id]["ctext"]
        summary=self.data.iloc[id]["text"]

        encoder=self.tokenizer(text,
            max_length=self.max_elength,
            padding="max_length",
            truncation=True,
            return_tensors="pt")

        decoder=self.tokenizer(summary,
            max_length=self.max_olength,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        return {
            "text":text,
            "summary":summary,
            "encoder_ids":encoder["input_ids"].squeeze(),
            "encoder_attention_mask":encoder["attention_mask"].squeeze(),
            "decoder_ids":decoder["input_ids"].squeeze(),
            "decoder_attention_mask":decoder["attention_mask"].squeeze()
        }

In [ ]:
model_name="google-t5/t5-base"

tokenizer=T5TokenizerFast.from_pretrained(model_name)
model=T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
train_dataset=NewsDataset(train_data,tokenizer,512,128)
test_dataset=NewsDataset(test_data,tokenizer,512,128)

In [ ]:
(len(train_dataset),len(test_dataset))

In [ ]:
train_dataset[0]

In [ ]:
test_dataset[0]

In [ ]:
train_dataloader=DataLoader(train_dataset,batch_size=8,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=8,shuffle=False)

In [ ]:
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.0005)

In [ ]:
torch.cuda.empty_cache()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()
with torch.no_grad():
    for _,batch in enumerate(test_dataloader):
        y=batch["decoder_ids"].to(device,dtype=torch.long)
        y_ids=y[:, :-1].contiguous()
        labels=y[:, 1:].clone().detach()
        labels[y[:, 1:]==tokenizer.pad_token_id]=-100
        ids = batch['encoder_ids'].to(device, dtype=torch.long)
        mask = batch['encoder_attention_mask'].to(device, dtype=torch.long)

        outputs=model(input_ids=ids,attention_mask=mask,decoder_input_ids=y_ids,labels=labels)
        loss=outputs[0]

        print(f"Batch:{_}")
        print(f"Loss:{loss}")
        if _==10:
            break


In [ ]:
epochs=15
torch.cuda.empty_cache()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer.zero_grad()

for epoch in range(epochs):

    model.train()
    print(f"Epoch:{epoch+1}")
    print()
    for _,batch in enumerate(train_dataloader):
        y=batch["decoder_ids"].to(device,dtype=torch.long)
        y_ids=y[:, :-1].contiguous()
        labels=y[:, 1:].clone().detach()
        labels[y[:, 1:]==tokenizer.pad_token_id]=-100
        ids = batch['encoder_ids'].to(device, dtype=torch.long)
        mask = batch['encoder_attention_mask'].to(device, dtype=torch.long)

        outputs=model(input_ids=ids,attention_mask=mask,decoder_input_ids=y_ids,labels=labels)
        loss=outputs[0]

        if _%100==0 and _>0:
            print(f"Step:{_}")
            print(f"Training Loss:{loss.item()}")
            print()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
with torch.no_grad():
    for _,batch in enumerate(test_dataloader):
        y=batch["decoder_ids"].to(device,dtype=torch.long)
        y_ids=y[:, :-1].contiguous()
        labels=y[:, 1:].clone().detach()
        labels[y[:, 1:]==tokenizer.pad_token_id]=-100
        ids = batch['encoder_ids'].to(device, dtype=torch.long)
        mask = batch['encoder_attention_mask'].to(device, dtype=torch.long)

        outputs=model(input_ids=ids,attention_mask=mask,decoder_input_ids=y_ids,labels=labels)
        loss=outputs[0]

        print(f"Batch:{_}")
        print(f"Loss:{loss}")
        if _==10:
            break

In [ ]:
test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=False)

In [ ]:
model.eval()
torch.cuda.empty_cache()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

predictions=[]
actuals=[]
for _,batch in enumerate(test_dataloader):
    y=batch["decoder_ids"].to(device,dtype=torch.long)
    y_ids=y[:, :-1].contiguous()
    labels=y[:, 1:].clone().detach()
    ids = batch['encoder_ids'].to(device, dtype=torch.long)
    mask = batch['encoder_attention_mask'].to(device, dtype=torch.long)

    generated_ids=model.generate(
        input_ids=ids,
        attention_mask=mask,
        min_length=20,
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
    )
    preds=[tokenizer.decode(curr_id,skip_special_tokens=True,
                            clean_up_tokenization_spaces=True) for curr_id in generated_ids]
    target=[tokenizer.decode(curr_id,skip_special_tokens=True,
                            clean_up_tokenization_spaces=True) for curr_id in labels]

    predictions.extend(preds)
    actuals.extend(target)
    if _%100==0 and _>0:
        break


In [ ]:
print("Predictions:")
print(predictions[1])
print("Actuals:")
print(actuals[1])

In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load
import evaluate

In [ ]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predictions, references=actuals)
print(results)